In [1]:
import psycopg2  
import pprint  

/home/nguyenhanh/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [1]:
import psycopg2
import pandas as pd
conn = psycopg2.connect(dbname='yalgdb', user='postgres',
                            password='password', host='localhost', port=5439)
cursor = conn.cursor()

cursor.execute("SELECT data.word, data.gender, COUNT(data.count) FROM (WordCountAttributes NATURAL JOIN WORDS) \
                AS data \
                WHERE data.gender IS NOT NULL \
                GROUP BY data.word, data.gender")
rows = cursor.fetchall()
conn.close() 

/home/nguyenhanh/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
counts_w_in_male = sum([row[2] for row in rows if row[1] == 'male'])
counts_w_in_female = sum([row[2] for row in rows if row[1] == 'female'])

In [3]:
print(counts_w_in_male, counts_w_in_female)

245930 441824


In [4]:
rows

[('90-летие', 'female', 1),
 ('j-й', 'male', 1),
 ('k-го', 'female', 9),
 ('k-го', 'male', 7),
 ('xx', 'male', 1),
 ('а', 'female', 3005),
 ('а', 'male', 1775),
 ('ааа', 'female', 2),
 ('ааа', 'male', 1),
 ('аааа', 'female', 1),
 ('ааааааааааааааааа', 'female', 1),
 ('аарон', 'male', 1),
 ('абай', 'female', 8),
 ('абай', 'male', 7),
 ('абакан', 'female', 2),
 ('абакан', 'male', 1),
 ('абама', 'female', 3),
 ('абама', 'male', 3),
 ('аббас', 'female', 3),
 ('аббас', 'male', 2),
 ('аббревиатура', 'female', 5),
 ('аббревиатура', 'male', 4),
 ('абд', 'female', 1),
 ('абдалла', 'male', 1),
 ('абдулла', 'female', 2),
 ('абдулла', 'male', 2),
 ('абдуллаев', 'male', 1),
 ('абдуллах', 'female', 2),
 ('абдуллах', 'male', 2),
 ('абдулов', 'female', 1),
 ('абдуль', 'male', 1),
 ('абдуррахман', 'male', 1),
 ('абзац', 'female', 2),
 ('абзац', 'male', 2),
 ('аби', 'female', 4),
 ('аби', 'male', 4),
 ('абитуриент', 'male', 1),
 ('абонемент', 'female', 7),
 ('абонемент', 'male', 1),
 ('абонент', 'female

In [5]:
vocab = (set(row[0] for row in rows))

In [6]:
i = 0
vocabs = {}
vocabs_F = {}
vocabs_M = {}
while i < len(rows)-1:
    if rows[i][0] == rows[i+1][0]:
        n = rows[i][2] + rows[i+1][2]
        Kh_nablyug = (rows[i][2] - n/2)**2 /(n/2) + (rows[i+1][2] - n/2)**2 /(n/2)
        vocabs.update({rows[i][0]:Kh_nablyug})
        vocabs_F.update({rows[i][0]:(rows[i][2]*2/n - 1)})
        vocabs_M.update({rows[i+1][0]:(rows[i+1][2]*2/n - 1)})
        i += 2
    else:
        i += 1

In [7]:
len(vocabs)

15006

In [29]:
import pandas as pd

df = pd.DataFrame([vocabs])

In [30]:
add = []
for i in df.iloc[0]:
    if i > 3.8:
        add.append("Y")
    else:
        add.append("N")

In [31]:
df.loc[len(df)] = add
df.loc[len(df)] = list(vocabs_F.values())
df.loc[len(df)] = list(vocabs_M.values())

In [32]:
df.head()

,k-го,а,ааа,абай,абакан,абама,аббас,аббревиатура,абдулла,абдуллах,...,яростный,ярость,ярош,ясно,ясность,ясный,яхта,яценюк,ячейка,ящик
0,0.25,316.506,0.333333,0.0666667,0.333333,0,0.2,0.111111,0,0,...,0,0.0769231,0,0,0.666667,12.3023,0,0,0,0.181818
1,N,Y,N,N,N,N,N,N,N,N,...,N,N,N,N,N,Y,N,N,N,N
2,0.125,0.257322,0.333333,0.0666667,0.333333,0,0.2,0.111111,0,0,...,0,-0.0769231,0,0,0.333333,0.534884,0,0,0,-0.0909091
3,-0.125,-0.257322,-0.333333,-0.0666667,-0.333333,0,-0.2,-0.111111,0,0,...,0,0.0769231,0,0,-0.333333,-0.534884,0,0,0,0.0909091


In [33]:
df = df.transpose()
df.head()

,0,1,2,3
k-го,0.25,N,0.125,-0.125
а,316.506,Y,0.257322,-0.257322
ааа,0.333333,N,0.333333,-0.333333
абай,0.0666667,N,0.0666667,-0.0666667
абакан,0.333333,N,0.333333,-0.333333


In [40]:
df.columns.tolist()

[0, 1, 2, 3]

In [51]:
agree = df[1] == 'Y'
female = df[2] > 0
male = df[3] > 0
vocabs_choice_F = df[agree & female]
vocabs_choice_M = df[agree & male]
print(vocabs_choice_F.index.tolist(), vocabs_choice_M.index.tolist())

['а', 'абонемент', 'абсолютный', 'ава', 'август', 'автоматически', 'автор', 'авторский', 'адрес', 'аксессуар', 'активный', 'акция', 'але', 'алина', 'аллергия', 'аналогичный', 'ананас', 'анастасия', 'ангел', 'ангелина', 'английский', 'анна', 'антикризисный', 'аня', 'апартамент', 'аппетит', 'апрель', 'аромат', 'архитектура', 'асадов', 'ассортимент', 'б', 'бабушка', 'бажання', 'бали', 'бальзам', 'бантик', 'бегать', 'беда', 'бежать', 'без', 'безумно', 'безумный', 'белый', 'белье', 'берег', 'беречь', 'бернский', 'беруть', 'бесить', 'бесконечный', 'бижутерия', 'билет', 'битва', 'благодарить', 'благодарность', 'благодарный', 'благодаря', 'блендер', 'блеск', 'близкий', 'блин', 'блюдо', 'бо', 'бог', 'богатство', 'более', 'болезнь', 'болеть', 'боль', 'больница', 'больной', 'больше', 'больший', 'большинство', 'большой', 'бороться', 'босоножка', 'бояться', 'брак', 'браслет', 'брать', 'бровь', 'бронь', 'бросать', 'бросить', 'буде', 'будто', 'будущий', 'будьня', 'букет', 'булгаков', 'булочка', 'буль

In [56]:
import pickle
with open('female.txt', 'wb') as F:
    pickle.dump(vocabs_choice_F.index.tolist(), F)
with open ('female.txt', 'rb') as F:
    L2 = pickle.load(F)
print(L2)
F.close()

['а', 'абонемент', 'абсолютный', 'ава', 'август', 'автоматически', 'автор', 'авторский', 'адрес', 'аксессуар', 'активный', 'акция', 'але', 'алина', 'аллергия', 'аналогичный', 'ананас', 'анастасия', 'ангел', 'ангелина', 'английский', 'анна', 'антикризисный', 'аня', 'апартамент', 'аппетит', 'апрель', 'аромат', 'архитектура', 'асадов', 'ассортимент', 'б', 'бабушка', 'бажання', 'бали', 'бальзам', 'бантик', 'бегать', 'беда', 'бежать', 'без', 'безумно', 'безумный', 'белый', 'белье', 'берег', 'беречь', 'бернский', 'беруть', 'бесить', 'бесконечный', 'бижутерия', 'билет', 'битва', 'благодарить', 'благодарность', 'благодарный', 'благодаря', 'блендер', 'блеск', 'близкий', 'блин', 'блюдо', 'бо', 'бог', 'богатство', 'более', 'болезнь', 'болеть', 'боль', 'больница', 'больной', 'больше', 'больший', 'большинство', 'большой', 'бороться', 'босоножка', 'бояться', 'брак', 'браслет', 'брать', 'бровь', 'бронь', 'бросать', 'бросить', 'буде', 'будто', 'будущий', 'будьня', 'букет', 'булгаков', 'булочка', 'буль

In [57]:
import pickle
with open('male.txt', 'wb') as F:
    pickle.dump(vocabs_choice_M.index.tolist(), F)
with open ('male.txt', 'rb') as F:
    L2 = pickle.load(F)
print(L2)
F.close()

['авто', 'автомат', 'автомобиль', 'аккаунт', 'алексей', 'аллах', 'армия', 'аудитория', 'аэропорт', 'аэропорт-сити', 'бей', 'боец', 'бой', 'борьба', 'босс', 'браузер', 'бугор', 'варить', 'вверх', 'взбить', 'ви', 'вика', 'виноватый', 'витамин', 'вишня', 'вовсе', 'возможный', 'возраст', 'вообще', 'вопрос', 'вспомнить', 'встроенный', 'вулкан', 'гоблин', 'давла', 'датчик', 'двигатель', 'дебальцево', 'диапазон', 'донецк', 'дорасти', 'достижение', 'жим', 'заем', 'заинтересованный', 'заценивать', 'защитник', 'заявка', 'звено', 'игра', 'игровой', 'игрок', 'инфо', 'квест', 'клан', 'классический', 'клуб', 'копировать', 'кредит', 'кредитный', 'кредитор', 'крест', 'криптовалюта', 'матч', 'миллиметр', 'миссия', 'мифрить', 'набрать', 'нажми', 'небес', 'ник', 'новороссия', 'одержать', 'окоп', 'ополченец', 'ополчение', 'оружие', 'отидый', 'очки', 'панфиловец', 'партнерский', 'пассив', 'перешедший', 'пиар', 'пиариться', 'победить', 'поздравление', 'поликлиника', 'полоска', 'посланник', 'прийнявый', 'про